# 본문 스크래핑

In [1]:
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import random
import urllib.request
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
# import chromedriver_auto


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument('headless') # 창 숨기기


In [2]:
# from user_agent import generate_user_agent, generate_navigator

# print(generate_user_agent(device_type='desktop'))
# print(generate_user_agent(os='win', device_type='desktop'))

# navigator = generate_navigator()

# random_user = generate_user_agent(os='win', device_type='desktop')
# print(random_user)

In [3]:
from selenium.webdriver.chrome.options import Options

# chrome_options = Options()
# chrome_options.add_argument(random_user)
driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = options)

driver.maximize_window()
driver.get("https://www.naver.com/")
driver.implicitly_wait(10)


In [4]:
url_df = pd.read_csv("./나무문화/2022/나무문화_링크_2022_6월.csv", encoding='UTF-8')
print(url_df.shape, url_df.columns)
url_df


(11967, 3) Index(['url', 'title', 'date'], dtype='object')


,url,title,date
0,https://blog.naver.com/ccalvados/222795576909,7월 개봉 예정 영화 10편! 신작 영화 추천,2022.06.30.
1,https://blog.naver.com/stepan5844/222795577522,[파니의 주간일기 #4] 6월 20일 ~ 6월 30일 / 캐논카메라 안녕!...,2022.06.30.
2,https://blog.naver.com/i_ipossible/222795577237,파주 출판도시 숙소 지지향에서 책에 둘러쌓인 북스테이 했어요,2022.06.30.
3,https://blog.naver.com/hyhan2028/222795577392,하남원데이미술 카페이볼브 아크릴화 체험권+음료,2022.06.30.
4,https://blog.naver.com/charnnant/222795577621,[마곡/발산] 비가 와도 가족/연인/친구와 같이 가면 좋은 실내...,2022.06.30.
...,...,...,...
11962,https://blog.naver.com/hsook0417/222754273924,신영복 평전,2022.06.01.
11963,https://blog.naver.com/flower_daltuium/2227542...,작은 화병 속의 틔움 숲 테라리움,2022.06.01.
11964,https://blog.naver.com/sohee10723/222754236890,"춘천여행 : 당일치기 데이트코스, 반나절만에 춘천 한바퀴...",2022.06.01.
11965,https://blog.naver.com/poyang84/222754272099,열세 번째 세계유산 영주 부석사를 가다,2022.06.01.


In [5]:
blog_links = url_df['url'].to_list()
print(len(blog_links))


11967


In [6]:
# df = pd.DataFrame(columns = ['url','본문'])
# df.to_csv("./나무문화/2022/나무문화_본문_2022_6월.csv", index=False, encoding='UTF-8')
# #df.to_excel("./나무문화/2020/나무문화_본문_2020_6월.xlsx", index=False, encoding='UTF-8')

In [7]:
df = pd.read_csv("./나무문화/2022/나무문화_본문_2022_6월.csv", encoding='UTF-8')
df

,url,본문
0,https://blog.naver.com/ccalvados/222795576909,"7월 개봉 예정 영화, 신작 영화추천\n신작 영화 추천! 극장 개봉 예정 영화 7월..."
1,https://blog.naver.com/stepan5844/222795577522,6월 20일\n지인이 파니를 위해\n카메라를 닦고 닦고 또 닦아\n완전 새거처럼 만...
2,https://blog.naver.com/i_ipossible/222795577237,2박 3일 파주여행을 하고 온 담담!\n이번 여행은 혼자 떠나기도했고\n요즘 바쁘다...
3,https://blog.naver.com/hyhan2028/222795577392,안녕하세요 연쓰입니다. 오늘은 좀 특별한 이색카페를 다녀왔어요.\n연쓰는 평소에 집...
4,https://blog.naver.com/charnnant/222795577621,⏰ 9:30 ~ 18:00\n🚘 주차 가능\n안녕하세요! 챠넷입니다 :D\n장마가 ...
...,...,...
9287,https://blog.naver.com/ichmount/222770079094,백제시대를 담아낸 국립 부여박물관\n올 여름은 많이 더울 것이라고 하는데 정말 부여...
9288,https://blog.naver.com/kspak56/222775056769,"경주 엑스포대공원, 경주타워\n경북 아이와 가볼만한곳\n경주 엑스포대공원 경주타워\..."
9289,https://blog.naver.com/bomeeson/222775056852,"""뿌!"" 하며 항해를 알리는 커다란 경적소리, 파란색 바다, 남쪽을 따라 나는 커피..."
9290,https://blog.naver.com/rirajjang9/222775054778,"문화일보의 [속보]를 읽어보았습니다.\n美 Fed,\n기준금리 0.75%p↑에 “올..."


In [8]:
len(df)

9292

In [9]:
# url_df.iloc[5866]

In [10]:
df.isnull().sum()

url    0
본문     3
dtype: int64

In [11]:
# # 이건 필요할때 쓰면 됨. 블로그가 닫혔거나 삭제되었을때 빈 본문을 가진 데이터프레임 추가.
# content_dic = {'url' : ["https://blog.naver.com/pony8253/222783289591"], '본문' : [None]}
# additional_df = pd.DataFrame(content_dic)
# df = df.append(additional_df, ignore_index = True)

In [15]:
#### 삭제/비공개 링크 인덱스도 매번 바꿔줄 것!!! ###
# contents = []
# old_blogs = []
hide_link = []
waitsec = random.uniform(4,4.5)

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

for i in tqdm(blog_links[len(df):]):
    content_dic = {'url' : [], '본문' : []}
    content_dic['url'].append(i)
    time.sleep(waitsec)
    driver.get(i)
    driver.implicitly_wait(10)
    time.sleep(waitsec)
    try:
        driver.switch_to.frame("mainFrame")
    except TimeoutException as e:
        print('시간 초과 오류 : ', e)
        content_dic['본문'].append('수집불가능')
    except UnexpectedAlertPresentException as e:
    ### 이 오류 해결함. 에러 코드 파일에서 직접 임포트하면 된다!
        print(e)
                                                                            # 여기도 바꿔야 함
        print('{}번째 게시글은 비공개로 전환되었거나 게시판이 바뀜.'.format(blog_links.index(i)))
        hide_link.append(i)
        time.sleep(1.5)
        driver.switch_to.frame('mainFrame')
    try:
        a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
        content_dic['본문'].append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
        content_dic['본문'].append(a)
        #old_blogs.append(i)
    additional_df = pd.DataFrame(content_dic)
    df = pd.concat([df,additional_df])
    df.reset_index(drop=True, inplace=True)
    #df.to_excel("./나무문화/2020/나무문화_본문_2020_6월.xlsx", index=False, encoding='UTF-8')
    print(blog_links.index(i), df['본문'].tolist().index(a))
    
    display.clear_output(wait=True) # 출력지우기
    # if blog_links.index(i) != df['본문'].tolist().index(a):
    #     break


KeyboardInterrupt: 

In [16]:
df

,url,본문
0,https://blog.naver.com/ccalvados/222795576909,"7월 개봉 예정 영화, 신작 영화추천\n신작 영화 추천! 극장 개봉 예정 영화 7월..."
1,https://blog.naver.com/stepan5844/222795577522,6월 20일\n지인이 파니를 위해\n카메라를 닦고 닦고 또 닦아\n완전 새거처럼 만...
2,https://blog.naver.com/i_ipossible/222795577237,2박 3일 파주여행을 하고 온 담담!\n이번 여행은 혼자 떠나기도했고\n요즘 바쁘다...
3,https://blog.naver.com/hyhan2028/222795577392,안녕하세요 연쓰입니다. 오늘은 좀 특별한 이색카페를 다녀왔어요.\n연쓰는 평소에 집...
4,https://blog.naver.com/charnnant/222795577621,⏰ 9:30 ~ 18:00\n🚘 주차 가능\n안녕하세요! 챠넷입니다 :D\n장마가 ...
...,...,...
10557,https://blog.naver.com/honeytteok_/222769168536,블로그 오랜만이다👋\n주간일기 쓰면서 다시 블로그 열심히해봐야겠다^^\n(네이버포인...
10558,https://blog.naver.com/twistraight/222769165696,1\n해가 떠 있는 시간에 집에 있었던 기념\n어쩌다보니 1월에 근무부서를 옮기게 ...
10559,https://blog.naver.com/eozini/222769165867,2022. 6.6~6.11\n남산 산책에 나섰다가 서울타워까지 성곽길 오르다보니 한...
10560,https://blog.naver.com/yoony700/222769166545,Wat Chalong\n70 หมู่ที่ 6 Thanon Chao Fah Tawa...


In [17]:
df.to_csv("./나무문화/2022/나무문화_본문_2022_6월.csv", index=False, encoding='UTF-8')